# Pré-Treinamento

## 1. Entropia antes e depois do pré-treino
Na aula passada você aprendeu a preparar os dados e fazer o pré-treino do modelo usando dados não rotulados. Vamos colocar isto em prática com novos dados e vamos medir o impacto do pré-treinamento. Compute a entropia do capítulo 6 do livro _The Time Machine_ com o modelo GPT sem nenhum pré-treinamento. Depois faça o pré-treino do modelo usando os cinco primeiros capítulos do mesmo livro e em seguida volte a computar a entropia do capítulo 6. Para isso, faça o seguinte:

- Recrie as classes para o modelo GPT e as funções do data loader.
- Instancie o GPTModel com 124M de parâmetros.
- Passe o texto do capítulo 6 pelo modelo e compute a entropia.
- Faça o pré-treino do modelo usando os 5 primeiros capítulos.
- Passe novamente o texto do capítulo 6 pelo modelo pré-treinado e compute a entropia e veja a diferença.

Execute a célula logo abaixo para fazer o download dos arquivos. Um arquivo contém o texto dos 5 primeiros capítulos do livro que você usará para o pré-treinamento e o outro arquivo possui o texto do capítulo 6 que você usará para computar a entropia. O conteúdo dos arquivos é armazenado nas variáveis `time_machine_ch01_ch05` e `time_machine_ch06`.

#### Download dos arquivos

In [ ]:
import requests
import re

def download_file_from_google_drive(file_url, destination):
    file_id = extract_google_drive_id(file_url)
    URL = "https://docs.google.com/uc?export=download&confirm=1"

    session = requests.Session()

    response = session.get(URL, params={"id": file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"id": file_id, "confirm": token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)


def extract_google_drive_id(url):
    url_pattern = r"/d/([a-zA-Z0-9_-]+)/view\?"
    response = re.search(url_pattern, url)
    if response:
        return response.group(1)
    return None


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)


# download dos arquivos
url_time_machine_ch01_ch05 = "https://drive.google.com/file/d/1nFeGmveao-WxYt5eAGjTnln2JUu4ugCr/view?usp=sharing"
download_file_from_google_drive(url_time_machine_ch01_ch05, "The Time Machine - ch01 to ch05.txt")

url_time_machine_ch06 = "https://drive.google.com/file/d/16q2ftm6B9wRBO4hhUEUSH8uU-6RgKdxy/view?usp=sharing"
download_file_from_google_drive(url_time_machine_ch06, "The Time Machine - ch06.txt")

# carrega o conteúdo dos arquivos nas variáveis
with open("The Time Machine - ch01 to ch05.txt", "r", encoding="utf-8") as f:
    time_machine_ch01_ch05 = f.read()

with open("The Time Machine - ch06.txt", "r", encoding="utf-8") as f:
    time_machine_ch06 = f.read()

#### Imports
Os imports necessários já estão todos aqui

In [ ]:
import os
import urllib
import tiktoken
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#### Dataset e dataLoader

In [ ]:
# RECRIE AS CLASSES DO DATASET E DATALOADER



#### GPTModel

In [ ]:
# RECRIE AS CLASSES PARA CRIAÇÃO DO MODELO GPT



#### Funções de treinamento e avaliação

In [ ]:
# RECRIE AS FUNÇÕES PARA O TREINAMENTO DO MODELO, VISTAS NA ÚLTIMA AULA
# train_model_simple E FUNÇÕES RELACIONADAS



#### Instanciação do GPT 124M e tokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

# INSTANCIE O GPT PASSANDO A CONFIGURAÇÃO ACIMA
# INSTANCIE TAMBÉM O TOKENIZER TIKTOKEN USANDO O GPT2 ENCONDING



#### Entropia com o modelo sem nenhum pré-treino

In [ ]:
# CARREGE O CAPÍTULO 6 NO DATA LOADER E COMPUTE A ENTROPIA (A LOSS USADA NO TREINO)
# PASSANDO O TEXTO PELO MODELO SEM TREINAMENTO



#### Pré-treino com os capítulos 1 a 5

In [ ]:
# Dados de treino e validção
train_ratio = 0.90
split_idx = int(train_ratio * len(time_machine_ch01_ch05))
train_data = time_machine_ch01_ch05[:split_idx]
val_data = time_machine_ch01_ch05[split_idx:]

# parâmetros de treinamento
num_epochs = 10
eval_freq = 5
eval_iter = 5
start_context = "The Time Machine."
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)


# CRIE OS DATA LOADERS PARA OS DADOS DE TREINO E VALIDAÇÃO DEFINIDOS ACIMA
# EM SEGUIDA, CHAME A FUNÇÃO DE TREINO PASSANDO OS PARÂMETROS DEFINIDOS ACIMA



#### Entropia do capítulo 6 após o pré-treino com os capítulos 1 a 5

In [ ]:
# PASSE NOVAMENTE O CAPÍTULO 6 PELO MODELO, AGORA PRÉ-TREINADO E COMPUTE A
# ENTROPIA NOVAMENTE E VEJA A DIFERENÇA



## 2. Entropia com pesos do Hugging Face
Crie uma nova instância do modelo GPT a partir dos pesos pré-treinados no Hugging Face e calcule novamente a entropia do texto do capítulo 6

In [ ]:
file_name = "gpt2-small-124M.pth"
url = f"https://huggingface.co/rasbt/gpt2-from-scratch-pytorch/resolve/main/{file_name}"

if not os.path.exists(file_name):
    urllib.request.urlretrieve(url, file_name)
    print(f"Downloaded to {file_name}")

BASE_CONFIG = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": True,        # Query-key-value bias
    "emb_dim": 768,
    "n_layers": 12,
    "n_heads": 12
}

# INSTANCIE O MODELO GPT COM A BASE_CONFIG DEFINIDA ACIMA
# (USE A MESMA CLASSE QUE VC DEFINIU NO INÍCIO DESTE NOTEBOOK)
# E CARREGUE OS PESOS A PARTIR DO ARQUIVO BAIXADO PELO COMANDO ACIMA



In [ ]:
# CRIE O DATA LOADER COM O TEXTO DO CAPÍTULO 6, PASSE PELO MODELO E COMPUTE A
# ENTROPIA

